In [2]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [3]:
# from google.colab import drive
# drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [2]:
# !nvidia-smi

In [5]:
model_checkpoint = 't5-base'
gdir = 'drive/My Drive/Colab Notebooks/t5-2021-06-21/'
model_name='sparql-translator-t5-2021-06-21'
model_path='./models/'+model_name
ds_path=gdir+'/dataset/lc-quad-wikidata-2021-06-21'

In [6]:
!mkdir models

In [7]:
from tqdm import tqdm

In [8]:
!pip install datasets transformers -qqq

     |████████████████████████████████| 245kB 13.5MB/s 
     |████████████████████████████████| 2.5MB 20.0MB/s 
     |████████████████████████████████| 122kB 42.9MB/s 
     |████████████████████████████████| 245kB 30.7MB/s 
     |████████████████████████████████| 901kB 42.3MB/s 
     |████████████████████████████████| 3.3MB 31.7MB/s 
ERROR: transformers 4.7.0 has requirement huggingface-hub==0.0.8, but you'll have huggingface-hub 0.0.10 which is incompatible.


In [9]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk
raw_datasets = load_from_disk(ds_path)

In [10]:
# Preprocessing

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [34]:
print(raw_datasets['test']['translation'][0]['sparql'])

select distinct ?sbj where [ ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 ]


In [12]:
print(raw_datasets['test']['translation'][0]['en'],tokenizer(raw_datasets['test']['translation'][0]['en']))

Who is the  {country} for {head of state} of {Mahmoud Abbas} {'input_ids': [2645, 19, 8, 3, 2, 17529, 2, 21, 3, 2, 3313, 13, 538, 2, 13, 3, 2, 329, 9, 107, 51, 1063, 26, 891, 4883, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [13]:
prefix = "translate English to Sparql: "

In [14]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry

100%|██████████| 38513/38513 [00:00<00:00, 920072.17it/s]


In [15]:
print(max_input_length, max_target_length)

248 224


In [16]:
source_lang = "en"
target_lang = "sparql"

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at drive/My Drive/Colab Notebooks/t5-2021-06-21/dataset/lc-quad-wikidata-2021-06-21/train/cache-a911a15367b2c41c.arrow


In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 38513
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 9547
    })
})

In [19]:
# Fine-tuning the model

In [20]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [21]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.586100,0.571864


TrainOutput(global_step=4815, training_loss=0.7263862427645136, metrics={'train_runtime': 4069.3647, 'train_samples_per_second': 9.464, 'train_steps_per_second': 1.183, 'total_flos': 7302800497771008.0, 'train_loss': 0.7263862427645136, 'epoch': 1.0})

In [25]:
trainer.save_model(model_path)

In [26]:
!ls -l --block-size=M {model_path}

total 852M
-rw-r--r-- 1 root root   1M Jun 21 20:35 config.json
-rw-r--r-- 1 root root 851M Jun 21 20:35 pytorch_model.bin
-rw-r--r-- 1 root root   1M Jun 21 20:35 special_tokens_map.json
-rw-r--r-- 1 root root   1M Jun 21 20:35 tokenizer_config.json
-rw-r--r-- 1 root root   2M Jun 21 20:35 tokenizer.json
-rw-r--r-- 1 root root   1M Jun 21 20:35 training_args.bin


In [29]:
!cp {model_path}/* drive/MyDrive/models/